<a href="https://colab.research.google.com/github/gowripreetham/SJSU_Pycaret/blob/main/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1 — confirm Python version (3.12 recommended)
import sys
print(sys.version)

# STEP 2 — upgrade core libraries
!pip install -U pip setuptools wheel scikit-learn>=1.4

# STEP 3 — install PyCaret (latest master)
!pip install -U git+https://github.com/pycaret/pycaret.git@master
!pip install -U numpy==1.26.4 optuna scipy pandas matplotlib seaborn lightgbm xgboost catboost


In [ ]:
from pycaret.datasets import get_data
import pandas as pd

# Load built-in dataset
data = get_data('boston')
print("✅ Dataset loaded successfully:", data.shape)
data.head()


In [ ]:
# Create a fake time index (monthly for simplicity)
data = data[['medv']].copy()   # ✅ lowercase
data['Date'] = pd.date_range(start='2000-01-01', periods=len(data), freq='M')
data = data.set_index('Date')
data = data['medv'].asfreq('M')

# Plot
data.plot(title='🏠 Boston Median House Value Over Time', figsize=(8,4))
data.tail()


In [ ]:
!pip install -U dask[complete]==2024.4.1
!pip install -U pycaret[full]  # makes sure sktime + dask integration matches


In [ ]:
from pycaret.time_series import *

s = setup(
    data=data,
    fh=12,                # Forecast 12 future months
    session_id=123,
    fold=3,
    use_gpu=True          # ✅ enables GPU
)


In [ ]:
# only evaluate 6 fast models
best = compare_models(include=['polytrend', 'naive', 'snaive', 'arima', 'auto_arima', 'ets'])



In [ ]:
# Plot the forecast
plot_model(best, plot='forecast')

# Forecast for 24 months into the future
plot_model(best, plot='forecast', data_kwargs={'fh': 24})

# Residuals
plot_model(best, plot='residuals')


In [ ]:
# Predict on holdout (test set)
holdout_pred = predict_model(best)
print(holdout_pred.head())

# Predict next 24 months
predict_model(best, fh=24)


In [ ]:
save_model(best, 'boston_timeseries_model')
loaded_model = load_model('boston_timeseries_model')


In [ ]:
check_stats()
check_stats(estimator=best)
